## 5. Temperature and Humidity Prediction

In [ ]:
# Prepare regression data
reg_features = ['CO2', 'TVOC', 'PM10', 'PM2.5', 'CO', 'LDR', 'O3']
X_reg = df[reg_features]
y_temp = df['Temp']
y_hum = df['Hum']

# Split data
X_train_reg, X_test_reg, y_train_temp, y_test_temp = train_test_split(X_reg, y_temp, test_size=0.2)
_, _, y_train_hum, y_test_hum = train_test_split(X_reg, y_hum, test_size=0.2)

# Normalize features
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

# Create and train temperature prediction model
print("\nTraining Temperature Prediction Model...")
temp_model = Sequential([
    Dense(128, activation='relu', input_shape=(len(reg_features),)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

temp_model.compile(optimizer='adam', loss='mse')
temp_history = temp_model.fit(
    X_train_reg_scaled,
    y_train_temp,
    epochs=50,
    validation_split=0.2,
    verbose=1
)

# Create and train humidity prediction model
print("\nTraining Humidity Prediction Model...")
hum_model = Sequential([
    Dense(128, activation='relu', input_shape=(len(reg_features),)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

hum_model.compile(optimizer='adam', loss='mse')
hum_history = hum_model.fit(
    X_train_reg_scaled,
    y_train_hum,
    epochs=50,
    validation_split=0.2,
    verbose=1
)

# Make predictions
temp_pred = temp_model.predict(X_test_reg_scaled).flatten()
hum_pred = hum_model.predict(X_test_reg_scaled).flatten()

# Calculate metrics
metrics = {
    'Temperature': {
        'MSE': mean_squared_error(y_test_temp, temp_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test_temp, temp_pred)),
        'R²': r2_score(y_test_temp, temp_pred),
        'MAE': mean_absolute_error(y_test_temp, temp_pred)
    },
    'Humidity': {
        'MSE': mean_squared_error(y_test_hum, hum_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test_hum, hum_pred)),
        'R²': r2_score(y_test_hum, hum_pred),
        'MAE': mean_absolute_error(y_test_hum, hum_pred)
    }
}

# Print metrics
for target, metric in metrics.items():
    print(f"\n{target} Prediction Metrics:")
    for name, value in metric.items():
        print(f"{name}: {value:.4f}")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Temperature predictions
ax1.scatter(y_test_temp, temp_pred, alpha=0.5)
ax1.plot([y_test_temp.min(), y_test_temp.max()],
         [y_test_temp.min(), y_test_temp.max()], 'r--')
ax1.set_title(f'Temperature Prediction (R² = {metrics["Temperature"]["R²"]:.4f})')
ax1.set_xlabel('Actual Temperature')
ax1.set_ylabel('Predicted Temperature')

# Humidity predictions
ax2.scatter(y_test_hum, hum_pred, alpha=0.5)
ax2.plot([y_test_hum.min(), y_test_hum.max()],
         [y_test_hum.min(), y_test_hum.max()], 'r--')
ax2.set_title(f'Humidity Prediction (R² = {metrics["Humidity"]["R²"]:.4f})')
ax2.set_xlabel('Actual Humidity')
ax2.set_ylabel('Predicted Humidity')

plt.tight_layout()
plt.show()